# OLCI Water constituents

    Version: 2.0
    Date:    10/04/2019
    Author:  Ben Loveday and Hayley Evers-King (Plymouth Marine Laboratory)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as free-to-use in the public domain, with no warranty.

This aim of this code is to allow you to visualise the in water components provided by the complex water processor over a given scene. It is recommended that you familiarise yourself with the OLCI_spatial_interrogation.ipynb code prior to using this.

In [1]:
%matplotlib inline

# libraries are imported here, and we can import any library with an alias that allows us easy access to them later.
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
import warnings
import cartopy.crs as ccrs
import cartopy.feature as cfeature
warnings.filterwarnings('ignore')

In [2]:
def flag_data_fast(flags_we_want, flag_names, flag_values, flag_data, flag_type='WQSF'):
    flag_bits = np.uint64()
    if flag_type == 'SST':
        flag_bits = np.uint8()
    elif flag_type == 'WQSF_lsb':
        flag_bits = np.uint32()
    
    for flag in flags_we_want:
        try:
            flag_bits = flag_bits | flag_values[flag_names.index(flag)]
        except:
            print(flag + " not present")
    
    return (flag_data & flag_bits) > 0

Now we will start our script, proper.

To run this script, you will need to point it to where your data is. If you keep your scripts and files in the same folder, you will not need to set the full path to the data file. However you may want to store things in different places and so it is good practice to be specific. To help to find your data, please complete the MYPATH variable below with the output generated by the /Configuration_Testing/Data_Path_Checker.ipynb Jupyter notebook in the Configuration_Testing folder. 

In [3]:
# e.g. MYPATH = os.path.join("C:/","Users","me","Desktop")
#MYPATH = "<please insert your path from Data_Path_Checker.ipynb here, removing the quotes and chevrons>"

MYPATH = os.path.join("C:\\","Users","docle","Desktop","ocean-olly_wekeo_changes")

Typically, we have three in water components to consider: chlorophyll, suspended sediment and CDOM. Teh complex water processor provides us with explicit estimates of the first two. For the last element, we use the absorption of coloured detrital and dissolved material at 443 nm; ADG443_NN.

In [4]:
#Location: 1: India, 2: Philippines, 3: Vietnam, 4: Malaysia, 5: Indonesia
Location='Philippines'
input_root    = os.path.join(MYPATH,'OLCI_test_data')
if Location == 'India':
    input_path    = 'S3A_OL_2_WFR____20171226T045629_20171226T045929_20171227T105453_0179_026_076_2700_MAR_O_NT_002.SEN3'
elif Location == 'Philippines':
    input_path    = 'S3A_OL_2_WFR____20180309T014459_20180309T014759_20180310T092937_0179_028_345_2880_MAR_O_NT_002.SEN3'
elif Location == 'Vietnam':
    input_path    = 'S3A_OL_2_WFR____20180701T022652_20180701T022952_20180702T092657_0179_033_046_2700_MAR_O_NT_002.SEN3'
elif Location == 'Malaysia':
    input_path    = 'S3A_OL_2_WFR____20180620T021454_20180620T021754_20180621T090056_0180_032_274_2880_MAR_O_NT_002.SEN3'
elif Location == 'Indonesia':
    input_path    = 'S3A_OL_2_WFR____20180321T013646_20180321T013946_20180322T072247_0179_029_131_3060_MAR_O_NT_002.SEN3'
      
file_name_chl = 'chl_nn.nc'
file_name_tsm = 'tsm_nn.nc'
file_name_cdom = 'iop_nn.nc'

file_name_geo = 'geo_coordinates.nc'

So, lets load in our positional data and our quality flags

REMEMBER: Not all of the standard L2 flags provided are relevant for the NN products.

In [5]:
GEO_file      = xr.open_dataset(os.path.join(input_root,input_path,file_name_geo))
LAT           = GEO_file.variables['latitude'][:]
LON           = GEO_file.variables['longitude'][:]
GEO_file.close()

In [6]:
flags_we_want = ['CLOUD', 'CLOUD_AMBIGUOUS', 'CLOUD_MARGIN', 'INVALID', 'COSMETIC', 'SATURATED', 'SUSPECT',
                 'HISOLZEN', 'HIGHGLINT', 'SNOW_ICE', 'AC_FAIL', 'WHITECAPS']

file_name_flags = 'wqsf.nc'
FLAG_file = xr.open_dataset(os.path.join(input_root,input_path,file_name_flags))
# get all the flag names
flag_names = FLAG_file['WQSF'].flag_meanings.split(' ')
# get all the flag bit values
flag_vals = FLAG_file['WQSF'].flag_masks
# get the flag field itself
FLAGS = FLAG_file.variables['WQSF'].data
FLAG_file.close()

# make the flag mask using the function we defined above "flag_data_fast"
flag_mask = flag_data_fast(flags_we_want, flag_names, flag_vals, FLAGS, flag_type='WQSF')
flag_mask = flag_mask.astype(float)
flag_mask[flag_mask == 0.0] = np.nan

Lets load in and flag our data and plot. We have discussed plotting extensively in previous scripts, so here we will proceeed directly to plotting and comparing the data. We are plotting an ENTIRE full resolution scene, which can be processor intensive to display, so we are going to print directly to file

In [7]:
%%capture

fig1 = plt.figure(figsize=(10, 10), dpi=300)

gs  = gridspec.GridSpec(1, 1)

#load CHL_NN
OLCI_file_CHL = xr.open_dataset(os.path.join(input_root,input_path,file_name_chl))
CHL_NN        = OLCI_file_CHL.variables['CHL_NN'][:].data
CHL_NN[np.isfinite(flag_mask)] = np.nan
OLCI_file_CHL.close()

#plot CHL_NN
m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree(central_longitude=0.0))
f1 = plt.pcolormesh(LON, LAT, np.ma.masked_invalid(CHL_NN), vmin=np.log10(0.01), vmax=np.log10(10), cmap=plt.cm.Greens)
m.coastlines(resolution='50m', color='black', linewidth=0.25)
#m.add_feature(cfeature.LAND, facecolor='0.75')
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.ylabels_right = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}
cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07, ticks=[np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(3),np.log10(10)]) 
cbar.ax.set_xticklabels(['0.01','0.1','0.5','1','5','10'], fontsize=20) 
cbar.set_label('Chlorophyll, mg.m$^{-3}$', fontsize=20)
plt.title('OLCI [CHL_NN]', fontsize=20)

fig1.savefig('OLCI_WATER_COMPONENTS_CHL_'+Location+'.png', bbox_inches='tight')

In [8]:
%%capture

fig1 = plt.figure(figsize=(10, 10), dpi=300)

gs  = gridspec.GridSpec(1, 1)

#load TSM_NN
OLCI_file_TSM = xr.open_dataset(os.path.join(input_root,input_path,file_name_tsm))
TSM_NN        = OLCI_file_TSM.variables['TSM_NN'][:].data
TSM_NN[np.isfinite(flag_mask)] = np.nan
OLCI_file_TSM.close()

#plot TSM_NN
m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree(central_longitude=0.0))
f1 = plt.pcolormesh(LON, LAT, np.ma.masked_invalid(TSM_NN), vmin=np.log10(0.01), vmax=np.log10(10), cmap=plt.cm.YlGnBu_r)
m.coastlines(resolution='50m', color='black', linewidth=0.25)
#m.add_feature(cfeature.LAND, facecolor='0.75')
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.ylabels_right = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}
cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07, ticks=[np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(3),np.log10(10)]) 
cbar.ax.set_xticklabels(['0.01','0.1','0.5','1','5','10'], fontsize=20) 
cbar.set_label('Total suspended material, g.m$^{-3}$', fontsize=20)
plt.title('OLCI [TSM_NN]', fontsize=20)

fig1.savefig('OLCI_WATER_COMPONENTS_TSM_'+Location+'.png', bbox_inches='tight')

In [9]:
%%capture

fig1 = plt.figure(figsize=(10, 10), dpi=300)

gs  = gridspec.GridSpec(1, 1)

#load ADG
OLCI_file_ADG = xr.open_dataset(os.path.join(input_root,input_path,file_name_cdom))
ADG_NN        = OLCI_file_ADG.variables['ADG443_NN'][:].data
ADG_NN[np.isfinite(flag_mask)] = np.nan
OLCI_file_ADG.close()

#plot ADG
m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree(central_longitude=0.0))
f1 = plt.pcolormesh(LON, LAT, np.ma.masked_invalid(ADG_NN), vmin=np.log10(0.01), vmax=np.log10(10), cmap=plt.cm.YlOrBr)
m.coastlines(resolution='50m', color='black', linewidth=0.25)
#m.add_feature(cfeature.LAND, facecolor='0.75')
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.ylabels_right = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}
cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07, ticks=[np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(3),np.log10(10)]) 
cbar.ax.set_xticklabels(['0.01','0.1','0.5','1','5','10'], fontsize=20) 
cbar.set_label('CDM absorption coefficient, m$^{-1}$', fontsize=20)
plt.title('OLCI [ADG_NN]', fontsize=20)

fig1.savefig('OLCI_WATER_COMPONENTS_ADG_'+Location+'.png', bbox_inches='tight')

<br> <a href="./13_OLCI_spectral_interrogation.ipynb"><< OLCI - Spectral Interrogation</a><span style="float:right;"><a href="./15_OLCI_CHL_comparison.ipynb">15 - Ocean and Land Colour Instrument - CHL algorithm comparison >></a>  <hr> <p style="text-align:left;">This project is licensed under the <a href="/edit/LICENSE">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eo-lab-usc-open/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:training@eumetsat.int>Contact</a></span></p>